Load Libraries

In [4536]:
import re
import PyPDF2
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
import cv2 as cv
import numpy as np
import pytesseract
from pytesseract import Output
import os
from icecream import ic
import pandas as pd
from datetime import datetime
from pdf2image import convert_from_path

Reference  : https://www.youtube.com/watch?v=oXlwWbU8l2o&t=1071s 

             https://www.youtube.com/watch?v=ADV-AjAXHdc

Load PYTesseract

In [4537]:
os.environ["PATH"] = r"C:\Users\nandan.chitale\Documents\Projects\R&D\PDF Data reader python\Poppler\Release-23.07.0-0\poppler-23.07.0\Library\bin"
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

#### Teserract Config

##### --psm: Page Segmentation Mode (PSM) controls how Tesseract interprets the image. It specifies the layout analysis mode. 

###### 0 = Orientation and script detection (OSD) only.
###### 1 = Automatic page segmentation with OSD.
###### 2 = Automatic page segmentation, but no OSD, or OCR. (not implemented)
###### 3 = Fully automatic page segmentation, but no OSD. (Default)
###### 4 = Assume a single column of text of variable sizes.
###### 5 = Assume a single uniform block of vertically aligned text.
###### 6 = Assume a single uniform block of text.
###### 7 = Treat the image as a single text line.
###### 8 = Treat the image as a single word.
###### 9 = Treat the image as a single word in a circle.
###### 10 = Treat the image as a single character.
###### 11 = Sparse text. Find as much text as possible in no particular order.
###### 12 = Sparse text with OSD.
###### 13 = Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific.

##### --oem: OCR Engine Mode (OEM) specifies the OCR engine to use.
###### 0 = Original Tesseract only.
###### 1 = Neural nets LSTM only.
###### 2 = Tesseract + LSTM.
###### 3 = Default, based on what is available.

##### --config: This parameter is used to specify additional Tesseract configuration variables. 

Save Original Image

In [4538]:
def saveOriginalImg(img, index, output_path, write=True):
    cv.imwrite(os.path.join(output_path, f"{index}_original.png"), img)
    return img

GraySacle Image

In [4539]:
def grayScaleImage(img, output_path, index, write=True):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_gray.png"), gray)
    return gray

Blur Grayscaled Image

In [4540]:
def blurImage(img, output_path, index, write=True):
    blur = cv.GaussianBlur(img, (5,5), 0)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_blur.png"), blur)
    return blur

Invert Grayscaled Image to read Lines accurately

In [4541]:
def thrasholdImage(img, output_path, index, write=True):
    thresh = cv.threshold(img, 0,255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]
    # thresh = cv.threshold(img, 127,255, cv.THRESH_BINARY)[1]
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_thresh.png"), thresh)
    return thresh

Invert Image

In [4542]:
def invertImage(img, output_path, index, write=True):
    invert = cv.bitwise_not(img)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_inverted.png"), invert)
    return invert

Erode Vertical Lines

In [4543]:
def erodeVerticalLines(img, output_path, index, write=True):
    hor = np.array([[1,1,1,1,1,1]])
    vertial_lines_eroded_image = cv.erode(img, hor, iterations=10)
    vertial_lines_eroded_image = cv.dilate(vertial_lines_eroded_image, hor, iterations=15)
    
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_verode.png"), vertial_lines_eroded_image)
    return vertial_lines_eroded_image

Erode Horizontal Lines

In [4544]:
def erodeHorizontalLines(img, output_path, index, write=True):
    ver = np.array([[1],[1],[1]])
    horizontal_lines_eroded_image = cv.erode(img, ver, iterations=15)
    horizontal_lines_eroded_image = cv.dilate(horizontal_lines_eroded_image, ver, iterations=15)
    
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_herode.png"), horizontal_lines_eroded_image)
    return horizontal_lines_eroded_image

Combine Vertical and Horizontal Eroded Images

In [4545]:
def combineErodedImages(vErodeImg, hErodeImg, output_path, index, write=True):
    combinedImages = cv.add(vErodeImg, hErodeImg)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_eroded_combined.png"), combinedImages)
    return combinedImages

Dialate Combined Images to make lines thicker

In [4546]:
def dialateImageToMakeLinesThicker(img, output_path, index, write=True):
    kernal = cv.getStructuringElement(cv.MORPH_RECT, (2,2))
    combineImageDialated = cv.dilate(img, kernal, iterations=1)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_combinedDialated.png"), combineImageDialated)
    return combineImageDialated

In [4547]:
def combineRemovedNoiseLineImage(imgWithoutLineNoise, imgWithLinesEnhanced, output_path, index, write=True):
    combinedImages = cv.add(imgWithoutLineNoise, imgWithLinesEnhanced)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_combineRemovedNoiseLineImage.png"), combinedImages)
    return combinedImages

Subtract combined and dilated image from original image

In [4548]:
def subtractCombinedDilatedFromOriginalImage(invertedImg, dilatedCombinedImg, output_path, index, write=True):
    imgWithoutLines = cv.subtract(invertedImg, dilatedCombinedImg)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_imgWithoutLines.png"), imgWithoutLines)
    return imgWithoutLines

Remove noise with erode and dilate

In [4549]:
def removeNoiseWithErodeAndDilate(img, output_path, index, write=True):
    kernal = cv.getStructuringElement(cv.MORPH_RECT, (2,2))
    imgWithoutLinesNoiseRemoved = cv.erode(img, kernal, iterations=1)
    kernal = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
    imgWithoutLinesNoiseRemoved = cv.dilate(imgWithoutLinesNoiseRemoved, kernal, iterations=2)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_removeNoiseWithErodeAndDilate.png"), imgWithoutLinesNoiseRemoved)
    return imgWithoutLinesNoiseRemoved

Dilate Image
(Pixels are added when structuring elements hits part of image)

In [4550]:
def dilateImage(thrasholdImg, output_path, index, write=True):
    dilation_1_iterations = 12
    dilation_2_iterations = 12
    var_kernel_size_to_remove_gaps_between_words = (4,2)
    var_simple_kernel_size = (1,5)
    # kernel_to_remove_gaps_between_words = cv.getStructuringElement(cv.MORPH_TOPHAT, var_kernel_size_to_remove_gaps_between_words)
    simple_kernel = np.ones(var_simple_kernel_size, np.uint8)
    dilated_image_1 = cv.dilate(thrasholdImg, var_kernel_size_to_remove_gaps_between_words, iterations=dilation_1_iterations)
    dilated_image = cv.dilate(dilated_image_1, simple_kernel, iterations=dilation_1_iterations)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_dilateImage1.png"), dilated_image_1)
        cv.imwrite(os.path.join(output_path, f"{index}_dilateImage2.png"), dilated_image)
        var_kernel_to_remove_gaps_between_words = f"kernel_to_remove_gaps_between_words = cv.getStructuringElement(cv.MORPH_DILATE, {var_kernel_size_to_remove_gaps_between_words})\n"
        var_simple_kernel = f"simple_kernel = np.ones({var_simple_kernel_size}, np.uint8)\n"
        var_dilation_1 = f"kernel_to_remove_gaps_between_words : {dilation_1_iterations} Iterations\n"
        var_dilation_2 = f"simple_kernel : {dilation_2_iterations} Iterations\n"
        with open(os.path.join(output_path, "kernel.config"), "w+") as file:
            file.writelines(var_kernel_to_remove_gaps_between_words)
            file.writelines(var_simple_kernel)
            file.writelines(var_dilation_1)
            file.writelines(var_dilation_2)
    return dilated_image

Find BLOB Contours

In [4551]:
def findContours(img, originalImg, output_path, index, write=True):
    result = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = result[0]
    
    imgWithControusDrawn = originalImg.copy()
    cv.drawContours(imgWithControusDrawn, contours, -1, (0, 255, 0), 1)
    if write:
        cv.imwrite(os.path.join(output_path, f"{index}_imgWithControusDrawn.png"), imgWithControusDrawn)
    return contours

Convert contours to bounding boxes

In [4552]:
def convert_contours_to_bounding_boxes(contours, original_Img, output_path, index, write=True):
    bounding_boxes = []
    image_with_all_bounding_boxes = original_Img.copy()
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        bounding_boxes.append((x, y, w, h))
        # This line below is about
        # drawing a rectangle on the image with the shape of
        # the bounding box. Its not needed for the OCR.
        # Its just added for debugging purposes.
        image_with_all_bounding_boxes = cv.rectangle(image_with_all_bounding_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        
    if write:
            cv.imwrite(os.path.join(output_path, f"{index}_image_with_all_bounding_boxes.png"), image_with_all_bounding_boxes)
            
    return bounding_boxes

get mean height of bounding box

In [4553]:
def get_mean_height_of_bounding_boxes(bounding_boxes):
    heights = []
    for bounding_box in bounding_boxes:
        x, y, w, h = bounding_box
        heights.append(h)
    return np.mean(heights)

sort_bounding_boxes_by_y_coordinate

In [4554]:
def sort_bounding_boxes_by_y_coordinate(bounding_boxes):
    bounding_boxes = sorted(bounding_boxes, key=lambda x: x[1])
    return bounding_boxes

Club bounding boxes by similar y co-ordinates into rows

In [4555]:
def club_all_bounding_boxes_by_similar_y_coordinates_into_rows(mean_height, bounding_boxes):
    rows = []
    half_of_mean_height = mean_height / 2
    current_row = [ bounding_boxes[0] ]
    for bounding_box in bounding_boxes[1:]:
        current_bounding_box_y = bounding_box[1]
        previous_bounding_box_y = current_row[-1][1]
        distance_between_bounding_boxes = abs(current_bounding_box_y - previous_bounding_box_y)
        if distance_between_bounding_boxes <= half_of_mean_height:
            current_row.append(bounding_box)
        else:
            rows.append(current_row)
            current_row = [ bounding_box ]
    rows.append(current_row)
    
    return rows

Sort all rows by x co-ordinates

In [4556]:
def sort_all_rows_by_x_coordinate(rows):
    for row in rows:
        row.sort(key=lambda x: x[0])
    return rows

Teserract Config

In [4557]:
tesseractConfig = r"--dpi 72 --psm 6 --oem 3 tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

Method to remove noise characters from string

Method to remove starting noise from string

In [4558]:
def remove_prefix_noise(input_string):
    try:
        pattern = r'^[\s|?/]+'
        result = re.sub(pattern, '', input_string)
        return result
    except Exception as e:
        ic(e)

Serach Result from py teserractx

In [4559]:
def get_result_from_tersseract(image_path):
    # output = subprocess.getoutput('tesseract ' + image_path + ' - -l eng --oem 3 --psm 7 --dpi 72 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789().calmg* "')
    
    # Convert text to boxes using teserract
    # output = pytesseract.image_to_boxes(image_path, lang='eng', config="--psm 7 --dpi 72")
    
    # Draw boxes around text generated by teserract
    # img = cv.imread(os.path.join(image_path))
    # height, width, _ = img.shape
    # teserractImg = img.copy()
        
    output = pytesseract.image_to_string(image_path, config=tesseractConfig, output_type=Output.STRING)
    # amount_boxes = len(data['text'])
    # for i in range(amount_boxes):
    #     if(float(data['conf'][i])) > 70:
    #         (x,y,width,height) = (data['left'][i],data['top'][i],data['width'][i],data['height'][i])
    #         cv.rectangle(teserractImg, (x,y), (x+width, y+height), (0,255,0), 2)
                
    # cv.imwrite(os.path.join(output_path, f"teserractImg.png"), teserractImg)
    output = output.strip()
    output = remove_prefix_noise(output)
    
    return output

Extracting text from box using OCR

In [4560]:
def crop_each_bounding_box_and_ocr(rows, original_image, image_slice_path):
    table = []
    current_row = []
    image_number = 0
    for row in rows:
        for bounding_box in row:
            x, y, w, h = bounding_box
            y = y - 5
            cropped_image = original_image[y:y+h, x:x+w]
            sliced_img_path = os.path.join(image_slice_path, f"{str(image_number)}.jpg")
            cv.imwrite(sliced_img_path, cropped_image)
            results_from_ocr = get_result_from_tersseract(sliced_img_path)
            current_row.append(results_from_ocr)
            image_number += 1
        table.append(current_row)
        current_row = []
        
    return table

Append data to excel

In [4561]:
def generate_excel_file(table, output_path, outputFileName = "output.xlsx")->bool:
    returnValue = False
    try:
        filePath = os.path.join(output_path, outputFileName)
        # Check file exists. if not,create one
        if not os.path.exists(filePath):
            # Create a new workbook
            workbook = Workbook()
        else:
            # Load workbook
            workbook = openpyxl.load_workbook(filePath)
            
        # Select active sheet
        worksheet = workbook.active
        
        # Append data
        for row in table:
            worksheet.append(row)
            
        # Save data
        workbook.save(filePath)
    except FileNotFoundError:
        workbook = openpyxl.Workbook()
    except Exception as e:
        ic(e)
    
    return returnValue

Draw Bounding Boxes around text in image

In [4562]:
def getBoundingBoxesAroundTextInImage(img, output_path):
    """
        This method is used to transform input image
        Which will be used for text detection using pytteserract
        Steps : 
            1> Input Image
            2> Grayscale
            3> Thrashhold (INVERT)
            4> Vertial lines erode
            5> Horizontal lines Erode
            6> Combine Vertial and horizontal lines erode
            7> Dilate Combined eroded lines image
            8> Combine thrashold with dilated lines image
            9> Remove lines from thrashold (8-3)
            10> Remove noise from image using erode and dilate
            11> Redilate noise reduced image
            12> Draw contoures around text
            13> Convert contours to bounding boxes.

    Args:
        img (Image(opencv)): Opencv image object
    """
    try:
        # Save Original Image
        originalImg = saveOriginalImg(img, 1, output_path)
        
        # Grayscale image
        img = grayScaleImage(img, output_path, index=2)

        # Thrashold Image
        thrasholdImg = thrasholdImage(img, output_path, index=3)

        # Invert image
        # invertedImg = invertImage(thrasholdImage, output_path,index=4)

        # Erode vertical lines
        vErodeImg = erodeVerticalLines(thrasholdImg, output_path, index=5)

        # Erode Horizontal lines
        hErodeImg = erodeHorizontalLines(thrasholdImg, output_path, index=6)

        # Combine Eroded Images
        combinedErodedImage = combineErodedImages(vErodeImg, hErodeImg, output_path, index=7)

        # Dilate Combined Image to make lines thicker
        linesDialatedImage = dialateImageToMakeLinesThicker(combinedErodedImage, output_path, index=8)

        # Remove the lines
        imgWithoutLines = subtractCombinedDilatedFromOriginalImage(thrasholdImg , linesDialatedImage, output_path, index=9)

        # Remove Noise with Erode and Dilate
        noiseRemovedImg = removeNoiseWithErodeAndDilate(imgWithoutLines, output_path, index=10)

        # Combine Removed noise image with lines image
        # img = combineRemovedNoiseLineImage(noiseRemovedImg, linesDialatedImage, output_path, index=11)

        # Dilate Image
        dilatedImage = dilateImage(noiseRemovedImg, output_path, index=12)

        # Contours
        contours = findContours(dilatedImage, originalImg, output_path, index=13)

        # Convert contours to boxes
        boundingBoxes = convert_contours_to_bounding_boxes(contours, originalImg, output_path, index=14)
        
        return boundingBoxes, noiseRemovedImg
    except Exception as e:
        ic(e)

Extract text from bounding boxes

In [4563]:
def extractTextUsingBoundingBoxes(boundingBoxes, img, image_slice_path):
    """ Method to get text from images with bounding boxes
        This method will create sub images for each bounding box
        Then will use pyttesseract to get the text from sub image
        And finally, will convert that data to excel sheet.
    Args:
        boundingBoxes (dict): Bounding boxes around text in image
        img (OPENCV image object): Original Image to draw bounding boxes on.
    """
    try:
        # Get mean height of boxes
        meanHeight = get_mean_height_of_bounding_boxes(boundingBoxes)

        # Sort bounding boxes
        boundingBoxes = sort_bounding_boxes_by_y_coordinate(boundingBoxes)

        # Club all bounding boxes by similar y co-ordinates
        rows = club_all_bounding_boxes_by_similar_y_coordinates_into_rows(meanHeight, boundingBoxes)

        # Sort rows by x co-ordinate
        rows = sort_all_rows_by_x_coordinate(rows)

        # Crop each box and extract text using ocr
        table = crop_each_bounding_box_and_ocr(rows, img, image_slice_path)
        
        return table
    except Exception as e:
        ic(e)

Convert pdf page to images

In [4564]:
def pdf2ImageConverter(pdf_file_path, image_file_path)->bool:
    """Method to convert pdf pages to images

    Args:
        pdf_file_path (string): PDF File path
        image_file_path (string): Output Image file path

    Returns:
        bool: True/False on success/failure
    """
    returnValue = False
    try:
        images = convert_from_path(pdf_file_path, dpi=300)
        if len(images) > 0:
            for i, image in enumerate(images):
                image_path = f"{image_file_path}/page_{i+1}.png"
                image.save(image_path, "png")

            returnValue = True
    except Exception as e:
        ic(e)
        
    return returnValue

## Driver Code
### Steps : 

1> Load pdf file.
2> Convert pdf file pages to image.
3> Extract text data from each image 
4> Append text data to Excel file.

Setup PATHS

In [4565]:
current_timestamp = f"{datetime.today().strftime('%d_%b_%Y_%H_%M')}"

# This path is used to store all data 
img_path = os.path.join("pdf2Images",current_timestamp)
if not os.path.exists(img_path):
    os.mkdir(img_path)

Load PDF

In [4566]:
# pdffilename = 'PO_63_12042023.pdf'
# pdffilename = 'IPCA-PARVATI-01.04.21.pdf'
pdf_file_path = os.path.join('TestFiles')
# pdf_file_path = os.path.join('TestFiles', pdffilename)

Extract Data from Each Image and Append it to excel

In [4567]:
# Iterate on images converted from pdf
inputPdfFileDirs = 'TestFiles'
pdfFiles = [file for file in os.listdir(inputPdfFileDirs)]

for file in pdfFiles:
    pdfDataPath = os.path.join(img_path, file)
    pdfFile = os.path.join(inputPdfFileDirs, file)
    if not os.path.exists(pdfDataPath):
        os.makedirs(pdfDataPath)
        
    # Setup paths
    # Images converted from pdf pages
    pdf2ImageDataPath = os.path.join(pdfDataPath, "pdf2ImgData")
    if not os.path.exists(pdf2ImageDataPath):
        os.mkdir(pdf2ImageDataPath)
        
    # Tmp dir where image processing will be stored
    tmp_img_path = os.path.join(pdfDataPath, "tmp")
    if not os.path.exists(tmp_img_path):
        os.mkdir(tmp_img_path)
    
    # Output files path
    output_path = os.path.join(pdfDataPath, "output")
    if not os.path.exists(output_path):
        os.mkdir(output_path)    
    
    # Convert pdf file to image
    pdf2ImageConvesion = pdf2ImageConverter(pdfFile, pdf2ImageDataPath)
    print(f"File to image conversion {'succeessfull' if pdf2ImageConvesion else 'failed'} at {pdf2ImageDataPath}")
        
    imageFiles = [file for file in os.listdir(pdf2ImageDataPath)]

    for i, file in enumerate(imageFiles):
        save_tmp_img_path = os.path.join(tmp_img_path, str(i+1))
        if not os.path.exists(save_tmp_img_path):
            os.mkdir(save_tmp_img_path)
            
        image_slice_path = os.path.join(save_tmp_img_path, "image_slices")
        if not os.path.exists(image_slice_path):
            os.mkdir(image_slice_path)
            
        # Read File using opencv
        img = cv.imread(os.path.join(pdf2ImageDataPath, file))
        
        # Get Bounds on image
        bounds, img = getBoundingBoxesAroundTextInImage(img, save_tmp_img_path)
        
        # Get Text from bounding boxes
        textData = extractTextUsingBoundingBoxes(bounds, img, image_slice_path)
        
        if textData is not None:
            # Append Data to excel   
            generate_excel_file(textData, output_path)
    

File to image conversion succeessfull at pdf2Images\09_Aug_2023_16_53\Chandan1.pdf\pdf2ImgData


File to image conversion succeessfull at pdf2Images\09_Aug_2023_16_53\Chandan2.pdf\pdf2ImgData
File to image conversion succeessfull at pdf2Images\09_Aug_2023_16_53\Kundan1.pdf\pdf2ImgData
File to image conversion succeessfull at pdf2Images\09_Aug_2023_16_53\Senior1.pdf\pdf2ImgData
File to image conversion succeessfull at pdf2Images\09_Aug_2023_16_53\Shriram1.pdf\pdf2ImgData
File to image conversion succeessfull at pdf2Images\09_Aug_2023_16_53\Shriram2.pdf\pdf2ImgData
File to image conversion succeessfull at pdf2Images\09_Aug_2023_16_53\Shriram3.pdf\pdf2ImgData
